## TODO

- add screwholes to plate

In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

import sys
sys.path.append('../')
from plate import Config, Shape, get_key_positions, get_key_hole_shape, get_keys
from functools import partial
import math

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


/home/ben/anaconda3/envs/jcq3/lib/python3.10/site-packages/quaternion/calculus.py:310: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def fd_indefinite_integral(f, t):


In [181]:
def get_screw_positions(config: Config) -> [(float, float)]:
    sp=[]
    x_centre = max(config.row_key_numbers)* config.rowSpacing*0.5
    row_holes = config.row_key_numbers
    print(row_holes)
    nrows = len(config.row_key_numbers)
    for col_num in range(0,nrows,1):    
        row_size = row_holes[col_num]
        centre = row_size* config.rowSpacing*0.5

        if centre != x_centre:
            x_trans = x_centre - centre
        else:
            x_trans = 0
            
        if row_size%2 ==0: #for rows with even key numbers
            print(col_num, row_size)
            for row_num in range(1,row_size, 2):
                hole_x_pos = config.rowSpacing*row_num+x_trans-0.5*config.rowSpacing
                hole_y_pos = config.columnSpacing*col_num+0.0*config.columnSpacing
                sp.append((hole_x_pos, hole_y_pos))
                
        else: # for rows with odd key numbers
            print(col_num, row_size)
            for row_num in range(1,row_size, 2):
                hole_x_pos = config.rowSpacing*row_num+x_trans-0.5*config.rowSpacing
                hole_y_pos = config.columnSpacing*col_num+0.0*config.columnSpacing
                sp.append((hole_x_pos, hole_y_pos))
    return sp

def make_plate(config:Config, holes=True) -> cq.Sketch:
    key_hole_shape = get_key_hole_shape(config)
    kp = get_key_positions(config)
    plate = get_base(config, kp, thickness=config.plateThickness, base_fillet=config.edgeFillet).cut(get_keys(kp, key_hole_shape, config))
    hole_psns= get_screw_positions(config)
    if holes:
        plate= plate.faces(">Z").workplane().pushPoints(hole_psns).hole(config.screwHoleDiamater)
    return plate


def get_base(config: Config, kp, thickness,base_fillet):

    foot_x, foot_y = (config.columnSpacing / 2 + config.switchHoleSize, config.rowSpacing / 2 +
                      config.switchHoleSize) if config.shape == Shape.LEAN else (config.switchHoleSize, config.switchHoleSize)
    base = cq.Sketch()
   
    base = base.push(kp.values())
    if config.shape == Shape.LEAN:
        base = base.rect(foot_x, foot_y)\
            .faces().clean().faces()\
            .wires().clean()

    w = base.val().offset2D(-2, 'intersection')[0].close() # negative offset to remove space between case and keys on outside of plate
    wp = cq.Workplane()
    wp=wp.add(w)
    base = wp.wires().toPending().extrude(thickness).edges('|Z').fillet(base_fillet)

    return base

In [2]:
from config import read_config_from_json

config_dict = """{
    "name": "Tbaum",
    "row_key_numbers": [
    2,
    8,
    6,
    3,
    1
    ],
    "columnSpacing": 19,
    "rowSpacing": 19,
    "plateThickness": 2,
    "screwHoleDiamater": 2.4,
    "switchHoleSize": 13.9,
    "shape": 0,
    "notched_keyhole": true,
    "caseHeight": 20,
    "caseGap": 0.6,
    "wallThickness": 1.6,
    "floorThickness": 3.0,
    "edgeFillet": 2,
    "bottomFillet": 0,
    "controller": "None",
    "controllerYOffset": 5,
    "controllerBoxThickness": 2
}"""

config = read_config_from_json(string=config_dict)


In [46]:
kp = get_key_positions(config)
base_fillet = 2
if True:

    foot_x, foot_y = (config.columnSpacing / 2 + config.switchHoleSize, config.rowSpacing / 2 +
                      config.switchHoleSize) if config.shape == Shape.LEAN else (config.switchHoleSize, config.switchHoleSize)
    base = cq.Sketch()
   
    base = base.push(kp.values())
    if config.shape == Shape.LEAN:
        base = base.rect(foot_x, foot_y)\
            .faces().clean().faces()\
            .wires().clean()

    w = base.val().offset2D(-2, 'intersection')[0].close() # negative offset to remove space between case and keys on outside of plate
    

    sk = cq.Sketch()

    sk.face(w).vertices().fillet(base_fillet).faces().wires().clean()

    wp = cq.Workplane()
    wp=wp.add(sk)



In [47]:
wp.extrude(2)

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [184]:

kp = get_key_positions(config)
plate = get_base(config, kp, thickness=config.plateThickness, base_fillet=config.edgeFillet)


In [194]:
basic_shape = plate.faces('>Z').wires().toPending().extrude(config.caseHeight)

In [191]:
plate.faces('>Z').wires()

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.02s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…